# Cloud Workshop Azure Databricks
## 14. MLflow Serving Models with Azure ML service

<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.4 29/06/2020

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## 0. Setup

1. Ensure you are using or create a cluster specifying 
  * **Databricks Runtime Version:** Databricks Runtime 5.0 or above
  * **Python Version:** Python 3
1. Install required libraries or if using Databricks Runtime 5.1 or above (but not Databricks Runtime for ML), run Cmd 6.
   1. Create required libraries.
      * Source **PyPI** and enter `mlflow[extras]`. This installs `mlflow` and all its dependencies.
      * Source **PyPI** and enter `azureml-sdk[databricks]`.
   1. Install the libraries into the cluster.
1. Attach this notebook to the cluster.

In [5]:
import datetime
now = datetime.datetime.now()
print(now)

2020-07-06 08:21:18.796819

In [6]:
import sys
sys.version

Out[2]: '3.7.3 (default, Mar 17 2020, 19:42:22) \n[GCC 5.4.0 20160609]'

## 1. Create or load an Azure ML Workspace

Before models can be deployed to Azure ML, you must create or obtain an Azure ML Workspace. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

In [9]:
import azureml
from azureml.core import Workspace

subscription_id = "AREMPLACER" #Votre ID Azure ML service
resource_group = "AzureML_DB-rg" # Le ressource groupe Azure ML service
workspace_name = "AzureML_DB" # Le nom Azure ML service
workspace_region = "west europe" # région Azure ML service

/databricks/python/lib/python3.7/site-packages/azureml/data/dataset_type_definitions.py:69: DeprecationWarning: FileType Enum is Deprecated in > 1.0.39. Use strings instead.
 category=DeprecationWarning)

In [10]:
workspace = Workspace.create(name = workspace_name,
                             location = workspace_region,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

## 2. Build an Azure Container Image for model deployment

### 2.1 Use MLflow to build a Container Image for the trained model

Use the `mlflow.azuereml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

In [13]:
# Récupération de l'ID du modèle depuis MLflow
run_id1 = "bb4648a5ff884c12a863de8664c267a3"
model_uri = "runs:/" + run_id1 + "/model"

### Génération image

In [15]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace,
                                                      model_name="diabetes",
                                                      image_name="diabetesimage",
                                                      description="Sklearn ElasticNet for Diabetes prediction",
                                                      synchronous=False)

Registering model diabetes
/databricks/python/lib/python3.7/site-packages/mlflow/azureml/__init__.py:202: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 tags=tags,
/databricks/python/lib/python3.7/site-packages/azureml/core/image/container.py:159: DeprecationWarning: ContainerImageConfig class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 base_image, base_image_registry, cuda_version=cuda_version)
/databricks/python/lib/python3.7/site-packages/mlflow/azureml/__init__.py:207: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 models=[registered_model])
Creating image
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:401: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = Image(workspace, id=image_id)

In [16]:
# Prévoir 6 minutes de temps de traitement
model_image.wait_for_creation(show_output=True)

Running.................................................................
Succeeded
Image creation operation finished for image diabetesimage:1, operation "Succeeded"

## 3. Deploy the model to "dev" using Azure Container Instances (ACI)

### 3.1 Create an ACI webservice deployment using the model's Container Image

Using the Azure ML SDK, deploy the Container Image for the trained MLflow model to ACI.

In [19]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = "diabetes-aci"
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

/local_disk0/tmp/1593438818267-0/PythonShell.py:5: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 import linecache
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:815: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = cls(None)

#### Déploiement ACI

In [21]:
# Prévoir 3 minutes pour déployer
dev_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

## 4. Query the ACI deployed model

### 4.1 Load diabetes dataset

In [24]:
from sklearn import datasets
diabetes = datasets.load_diabetes()

### 4.2 Create sample input vector

In [26]:
import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

Out[10]: [0]

### 4.3 Evaluate the sample input vector by sending an HTTP request
Query the ACI webservice's scoring endpoint by sending an HTTP POST request that contains the input vector.

In [28]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

Scoring URL :

In [30]:
dev_webservice.scoring_uri

Out[12]: 'http://c926d513-4cf8-440c-a450-a5b02caecce0.westeurope.azurecontainer.io/score'

Appel du modèle :

In [32]:
dev_prediction = query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [201.3209729614444]

## 5. Deploy the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/). <br> Option 1 or Option 2.

### Option 1: Create a new AKS cluster

If you do not have an active AKS cluster for model deployment, create one using the Azure ML SDK.

> ##### Prévoir 6 minutes de temps de traitement

In [35]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "diabetes-aks" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating............................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None

### Option 2: Connect to an existing AKS cluster

If you already have an active AKS cluster running, you can add it to your Workspace using the Azure ML SDK.

In [37]:
#from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource group from https://porta..azure.com -> Find your resource group
#resource_group = "<resource-group>"

# Give the cluster a local name
#aks_cluster_name = "diabetes-cluster"

# Attatch the cluster to your workgroup
#attach_config = AksCompute.attach_configuration(resource_group=resource_group, cluster_name=aks_cluster_name)
#aks_target = ComputeTarget.attach(workspace, name="diabetes-compute", attach_config)

# Wait for the operation to complete
#aks_target.wait_for_completion(True)
#print(aks_target.provisioning_state)
#print(aks_target.provisioning_errors)

### 6. Deploy to the model's image to the specified AKS cluster

In [39]:
from azureml.core.webservice import Webservice, AksWebservice

prod_webservice_name = "diabetes-model-aks"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

/local_disk0/tmp/1593438818267-0/PythonShell.py:10: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 import six
/databricks/python/lib/python3.7/site-packages/azureml/core/image/image.py:815: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
 image = cls(None)

#### Déploiement

In [41]:
# Prévoir 2 minutes de temps de traitement
prod_webservice.wait_for_deployment(show_output = True)

Running...................
Succeeded
AKS service creation operation finished, operation "Succeeded"

## 7. Query the deployed model in production

#### Evaluate the sample input vector by sending an HTTP request
Query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. Include this key in the HTTP request header.

In [44]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

#### Scoring URL :

In [46]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

### Test URL :

In [48]:
prod_prediction1 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'], 'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]]}
Received response: [201.32097296144437]

### Delete

In [50]:
#dev_webservice.delete()

In [51]:
#prod_webservice.delete()

In [52]:
#aks_target.delete()